Well be giving access to IoT things on a bucket, as described [here](https://docs.aws.amazon.com/iot/latest/developerguide/authorizing-direct-aws.html)

Get ready for a hell of a ride!

In [ ]:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import boto3
import base64, json
import requests

In [ ]:
region = boto3.session.Session().region_name
account_id = boto3.resource('iam').CurrentUser().arn.split(':')[4]
print(region)
print(account_id)

## Part 1

Role: defines effect (deny/allow), action (what is denied/allowed), resource (..on what)
RoleAlias: a copy of a Role that is only valid for a certain time

Let's create a Role that allows the creation of security tokens (sts=security token service) by using the credentials service.

I *think* that the entities / logic is this:
```

+--------------------------+  -----> gives creds, according to a role alias
|IoT credentials provider  |         "my-role-alias"  ---> creds allows stuff according to 
|server                    |                                        "my-role-credentials"
|                          |
+--------------------------+  <----- request creds according to role alias
  - can give creds requested         "my-role-alias" --link--> role "my-role-credentials"
    with certs because role          creds are requested with certificates
    because role alias
    "my-role-alias" has
    "it:AssumeRoleWithCertificate"
    enabled

  - can give creds for "my-role-alias"
    in the first place, since "my-role-alias"
    role alias "inherits" role "my-role-credentials"

```

Go to [aws roles](https://console.aws.amazon.com/iamv2/home#/roles)

Create a role (let's call it ``CREDENTIALS_ROLE``) and in the roles "Trust Relationship" tab, put this:
```
{
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }
} 
```
A quote from [here](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) says "Returns a set of temporary security credentials that you can use to access AWS resources that you might not normally have access to".

And from [here](https://docs.aws.amazon.com/iot/latest/developerguide/authorizing-direct-aws.html): "Configure the IAM role that the credentials provider assumes on behalf of your device"

The user that's hacking all this, needs these additional permissions:
```
{
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": [
            "iam:GetRole",
            "iam:PassRole"
        ],
        "Resource": "arn:aws:iam::263211xxxxxx:role/CREDENTIALS_ROLE"
    }
}
```
So go to [aws users](https://console.aws.amazon.com/iamv2/home#/users) & add an inline policy

==> now user has permission to pass this role to other users/services & create aliases
==> user can pass the ``CREDENTIALS_ROLE``

In [ ]:
role_name = "my-role-credentials"
credentials_role_arn = "arn:aws:iam::{account_id}:role/{role_name}".format(account_id=account_id, role_name=role_name)
credentials_role_alias = "my-role-alias"
print("use this inline policy for users:")
print("""{
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": [
            "iam:GetRole",
            "iam:PassRole"
        ],
        "Resource": "%s"
    }
}""" % (credentials_role_arn))

Create a "temporary copy" of the Role, aka "Role Alias":

In [ ]:
iot_client = boto3.client('iot')
try:
    response = iot_client.create_role_alias(
        roleAlias=credentials_role_alias,
        roleArn=credentials_role_arn,
        credentialDurationSeconds=900
    )
except Exception as e:
    print("got exception '"+str(e)+"'")
    print("this is still active?")

We still have to "attach" that role to the IoT device certificate in question, so go to AWS IoT => Manage => Things

There choose a thing and go to it's certificates.  There use "attach policies" and create this policy:
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": "iot:AssumeRoleWithCertificate",
      "Resource": "arn:aws:iot:us-west-2:263211956751:rolealias/CREDENTIALS_ROLE_ALIAS"
    }
  ]
}
```

In [ ]:
print("use this policy for the certificate:")
print("""{"Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": "iot:AssumeRoleWithCertificate",
      "Resource": "arn:aws:iot:%s:%s:rolealias/%s"
    }
  ]
}""" % (region, account_id, credentials_role_alias))

print("here is your link to manage iot:")
print("https://{region}.console.aws.amazon.com/iot/home?region={region}#/thinghub".format(region=region))

All set!  Let's request authentication tokes

In [ ]:
response = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)
print(response["endpointAddress"])
end_point_adr = response["endpointAddress"]

In [ ]:
cert_file ="../thing1/cert/thing1.cert.pem"
ca_cert_file ="../thing1/cert/root-CA.crt"
cert_pk_file="../thing1/cert/thing1.private.key"

your_thing_name="thing1"

cred_endpoint="https://"+end_point_adr+"/role-aliases/"+credentials_role_alias+"/credentials"
print(cred_endpoint)
resp = requests.get(
    cred_endpoint,
    cert=(cert_file, cert_pk_file, ca_cert_file), # files!
    headers={
        "x-amzn-iot-thingname":your_thing_name
    }
)

In [ ]:
print(resp)

In [ ]:
creds = json.loads(resp.content)["credentials"]

Congrats, credentials to access various AWS services, based on the IoT device credentials have been obtained!

## Part 2

In [ ]:
s3_client = boto3.resource('s3',
    aws_access_key_id=creds["accessKeyId"],
    aws_secret_access_key=creds["secretAccessKey"],
    aws_session_token=creds['sessionToken'])

Need to access a bucket?  You still need to grant your new role bucket rights, so head again to [IAM management](https://console.aws.amazon.com/iamv2/home#/roles/) and add a permission like this:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": "arn:aws:s3:::BUCKET_NAME"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": "arn:aws:s3:::BUCKET_NAME/*"
        }
    ]
}
```

In [ ]:
bucket_name = "my-test-bucket"

In [ ]:
print("use this permission:")
print("""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": "arn:aws:s3:::%s"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": "arn:aws:s3:::%s/*"
        }
    ]
}""" % (bucket_name, bucket_name))

In [ ]:
b = s3_client.Bucket(bucket_name)
for obj in b.objects.all():
    print(obj)

Role aliases can be listed and removed:

In [ ]:
lis = False
if lis:
    response = iot_client.list_role_aliases()
    print(response)

In [ ]:
rem = False
if rem:
    for name in response["roleAliases"]:
        iot_client.delete_role_alias(roleAlias=name)